In [1]:
import xarray as xr
from xarray.groupers import TimeResampler
import dask as da
from pandas import to_datetime
from dask.diagnostics import ProgressBar
from dask import persist
import time
from distributed import Client,LocalCluster

In [2]:
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 63.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49575,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 63.25 GiB
Comm: tcp://127.0.0.1:49609,Total threads: 4
Dashboard: http://127.0.0.1:49612/status,Memory: 10.54 GiB
Nanny: tcp://127.0.0.1:49578,


In [4]:
date_i = '2006-01-01'
date_f = '2006-01-05'


latN = 90
latS = 20
lonW = -150
lonE = -40

Var = '2m_temperature'

slice_xy = True
slice_t = True

In [5]:
def roll_longitude(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

def slice_temporal(ds,datei,datef):
    return ds.sel(time=slice(date_i,date_f))

def slice_spatial(ds,latN,latS,lonW,lonE):
    return ds.sel(latitude = slice(latN,latS),longitude=slice(lonW,lonE))

In [6]:
reanalysis = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks={"time": 48},
    #chunks = None,
    storage_options=dict(token='anon'),
)

In [7]:
pre_data = reanalysis

if slice_t:
    pre_data = slice_temporal(pre_data,date_i,date_f)

pre_data = roll_longitude(pre_data)

if slice_xy:
    pre_data = slice_spatial(pre_data,latN,latS,lonW,lonE)

data = pre_data[Var]

In [8]:
data

<xarray.DataArray '2m_temperature' (time: 120, latitude: 281, longitude: 441)> Size: 59MB
dask.array<getitem, shape=(120, 281, 441), dtype=float32, chunksize=(48, 281, 441), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 960B 2006-01-01 ... 2006-01-05T23:00:00
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [11]:
grouped_data = data.resample(time="D")
grouped_data


<DataArrayResample, grouped over 1 grouper(s), 5 groups in total:
    '__resample_dim__': 5 groups with labels 2006-01-01, ..., 2006-01-05>

In [12]:
daily_t2_max = grouped_data.max()
daily_t2_min = grouped_data.min()
daily_t2_min

<xarray.DataArray '2m_temperature' (time: 5, latitude: 281, longitude: 441)> Size: 2MB
dask.array<transpose, shape=(5, 281, 441), dtype=float32, chunksize=(2, 281, 441), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * time       (time) datetime64[ns] 40B 2006-01-01 2006-01-02 ... 2006-01-05
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [13]:
output_data = xr.Dataset({
        'daily_t2_max': daily_t2_max,
        'daily_t2_min': daily_t2_min
})

In [14]:
output_data

<xarray.Dataset> Size: 5MB
Dimensions:       (latitude: 281, longitude: 441, time: 5)
Coordinates:
  * latitude      (latitude) float32 1kB 90.0 89.75 89.5 ... 20.5 20.25 20.0
  * longitude     (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * time          (time) datetime64[ns] 40B 2006-01-01 2006-01-02 ... 2006-01-05
Data variables:
    daily_t2_max  (time, latitude, longitude) float32 2MB dask.array<chunksize=(2, 281, 441), meta=np.ndarray>
    daily_t2_min  (time, latitude, longitude) float32 2MB dask.array<chunksize=(2, 281, 441), meta=np.ndarray>

In [13]:
data.groupby("time.dayofyear").min(dim='time').compute()

<xarray.DataArray '2m_temperature' (dayofyear: 2, latitude: 281, longitude: 441)> Size: 991kB
array([[[241.77954, 241.77954, 241.77954, ..., 241.77954, 241.77954,
         241.77954],
        [241.70456, 241.70609, 241.70914, ..., 241.92494, 241.92494,
         241.92494],
        [241.75047, 241.75352, 241.75964, ..., 242.17136, 242.17136,
         242.17136],
        ...,
        [295.83014, 295.82098, 295.79956, ..., 296.44235, 296.46835,
         296.46835],
        [295.88525, 295.85922, 295.80872, ..., 296.59235, 296.60306,
         296.5847 ],
        [295.90363, 295.847  , 295.77505, ..., 296.72702, 296.71326,
         296.6658 ]],

       [[242.28233, 242.28233, 242.28233, ..., 242.28233, 242.28233,
         242.28233],
        [242.10995, 242.113  , 242.11911, ..., 242.8666 , 242.86507,
         242.86507],
        [242.12369, 242.1313 , 242.14198, ..., 243.58813, 243.58813,
         243.58508],
        ...,
        [295.36908, 295.30652, 295.0548 , ..., 295.8923 , 295.9625 ,
         296.26605],
        [295.2501 , 295.186  , 295.244  , ..., 296.06012, 296.2691 ,
         296.38046],
        [295.4728 , 295.46213, 295.46976, ..., 296.24164, 296.38962,
         296.40335]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * dayofyear  (dayofyear) int64 16B 1 2
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [ ]:
def calculate_min(ds):
    if len(ds['time']) == 0:
        return ds
    temp = ds.groupby("time.dayofyear").min(dim='time') 
    return temp

ds['new_var'] = ds['var_name'] * arr_2d

In [ ]:
file_name = "ERA5/arco-era5/data/teste.nc"

with ProgressBar():
    #tmin, tmax = da.compute(data.groupby("time.dayofyear").min(dim="time"), data.groupby("time.dayofyear").max(dim="time"))
    data.to_netcdf(file_name,compute=False).compute()

In [4]:
file_name = "ERA5/arco-era5/data/teste2.nc"

ds = xr.open_dataset(file_name,chunks={"time": 48})
#xr.open_mfdataset('my/files/*.nc', parallel=True)

ds

<xarray.Dataset> Size: 4GB
Dimensions:         (latitude: 281, time: 8760, longitude: 441)
Coordinates:
  * latitude        (latitude) float32 1kB 90.0 89.75 89.5 ... 20.5 20.25 20.0
  * time            (time) datetime64[ns] 70kB 2005-01-01 ... 2005-12-31T23:0...
  * longitude       (longitude) float32 2kB -150.0 -149.8 ... -40.25 -40.0
Data variables:
    2m_temperature  (time, latitude, longitude) float32 4GB dask.array<chunksize=(48, 281, 441), meta=np.ndarray>

In [5]:
tmin = ds.groupby("time.dayofyear").min(dim='time') #19.8 s


In [7]:
tmin.to_netcdf("ERA5/arco-era5/data/teste3.nc")

In [20]:
tmin = data.groupby("time.dayofyear").min(dim='time')
tmin = tmin.to_netcdf(file_name,compute=False)

In [21]:
%%time
tmin.compute()

CPU times: total: 1.88 s
Wall time: 2min 59s


In [13]:
#pull from 1 location
#data.sel(longitude=-45, latitude=-120, method='nearest').mean(dim='time').compute()

<xarray.DataArray '2m_temperature' ()> Size: 4B
np.float32(298.15717)
Coordinates:
    latitude   float32 4B 20.0
    longitude  float32 4B -45.0

In [16]:
file_name = "ERA5/arco-era5/data/teste.nc"
lazy = data.groupby("time.dayofyear")

In [20]:
grouped_data =lazy.max("time").persist()

In [ ]:
.max("time").to_netcdf(file_name,compute=False)

In [44]:
#map_blocks approach
ds_rechunk = data.chunk({'time': -1, 'latitude': 3})
ds_rechunk

def calculate_anomaly(ds):
    # needed to workaround xarray's check with zero dimensions
    # https://github.com/pydata/xarray/issues/3575
    if len(ds['time']) == 0:
        return ds
    gb = ds.groupby("time.dayofyear")
    clim = gb.min(dim='time')
    return clim

t2mmax_anom = xr.map_blocks(calculate_anomaly, ds_rechunk)
t2mmax_anom.data
%time t2mmax_std = t2mmax_anom.min(dim='time').compute()


dask.array<<this-array>-calculate_anomaly, shape=(8760, 281, 441), dtype=float32, chunksize=(8760, 3, 441), chunktype=numpy.ndarray>

In [35]:
#rechunck approach
day = data.time.dt.strftime("%h-%d").rename("day")
newchunks = xr.ones_like(day).astype(int).resample(time="M").count()
rechunked = data.chunk(time=tuple(newchunks.data))
import flox.xarray

tpm = flox.xarray.xarray_reduce(rechunked, rechunked.time.dt.dayofyear, 
                                func="min", 
                                method="cohorts")

t0 = time.time()
tpm = tpm.compute()
print(f'Yep: {time.time() - t0:.1f}s')

c:\Users\ls2236\AppData\Local\anaconda3\envs\arco\Lib\site-packages\xarray\groupers.py:403: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


In [36]:
#flox method
import flox.xarray

tpm = flox.xarray.xarray_reduce(data, data.time.dt.dayofyear, 
                                func="min", 
                                method="map_reduce")

t0 = time.time()
tpm = tpm.compute()
print(f'Yep: {time.time() - t0:.1f}s')

<xarray.DataArray '2m_temperature' (dayofyear: 365, latitude: 281,
                                    longitude: 441)> Size: 181MB
dask.array<transpose, shape=(365, 281, 441), dtype=float32, chunksize=(31, 281, 441), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 1kB 90.0 89.75 89.5 89.25 ... 20.5 20.25 20.0
  * longitude  (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * dayofyear  (dayofyear) int64 3kB 1 2 3 4 5 6 7 ... 360 361 362 363 364 365
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

Yep: 177.3s
